# Fall Detection using SisFall Dataset
Daniela Dias, nMec 98039

In [1]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

2025-05-07 13:24:02.293803: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746620642.394625     978 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746620642.424218     978 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1746620642.693166     978 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1746620642.693195     978 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1746620642.693197     978 computation_placer.cc:177] computation placer alr

Num GPUs Available:  1


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold
import tensorflow as tf
from sklearn.utils import compute_class_weight
from tensorflow.keras import layers, models, callbacks
import optuna
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

In [3]:
# Set random seed for reproducibility
np.random.seed(42)

## Deep Learning Preprocessing

This pipeline prepares raw time-series sensor data for deep learning models such as CNNs or LSTMs.

1. Raw sensor data is segmented into fixed-length overlapping windows (e.g., 4 seconds = 800 samples at 200 Hz).
    - Each window is treated as a single sample.
   - The overlap can be adjusted (e.g., 50% overlap).
2. Each window is converted into a 3D tensor: (number of samples, window length, number of channels).
    - For example, with 400-sample windows and 9 channels (3 sensors × 3 axes), the shape should be (N, 400, 9).
3. Labels are assigned to each window based on the most frequent class label within the window.
4. Input windows are normalized using z-score standardization based on the training set.
5. Subject-wise splitting is used to ensure no overlap of subjects between training and testing sets.

### Avoiding Subject Bias in Train-Test Split

When working with the SisFall dataset (or any dataset where multiple recordings come from the same individuals), it is essential to avoid data leakage caused by random sample splitting.

If we randomly split the dataset into training and testing sets, we risk placing samples from the same subject in both sets. This introduces what is known as "subject bias" — the model can inadvertently learn personal characteristics or movement patterns of specific individuals rather than learning to generalize fall detection across new, unseen people. This would artificially inflate evaluation metrics (such as accuracy and F1-score), because the model is partially memorizing rather than generalizing.

To address this, we use a subject-wise splitting strategy:  
- We first extract the list of unique subjects.  
- Then we split these subjects into train and test groups.  
- Finally, we assign samples based on the subject to which they belong.

This ensures that the model is evaluated on entirely unseen individuals, simulating real-world scenarios where fall detection must work for new users. It leads to more honest and generalizable performance metrics.

In [4]:
# Load the dataset from CSV file
sisfall_data = pd.read_csv('reduced_sisfall_dataset.csv')

In [5]:
# Check the first few rows of the dataset
sisfall_data.head()

,accel_adxl345_x,accel_adxl345_y,accel_adxl345_z,gyro_itg3200_x,gyro_itg3200_y,gyro_itg3200_z,accel_mma8451q_x,accel_mma8451q_y,accel_mma8451q_z,label,filename,subject
0,0.066406,-0.699219,-0.386719,-1.098633,-30.761719,-21.484375,0.074219,-0.680664,-0.272461,adl,D01_SA01_R01.txt,SA01
1,0.058594,-0.679688,-0.351562,-3.234863,-34.667969,-18.676758,0.046875,-0.659180,-0.248047,adl,D01_SA01_R01.txt,SA01
2,0.003906,-0.687500,-0.316406,-5.126953,-37.414551,-16.540527,-0.001953,-0.652344,-0.215820,adl,D01_SA01_R01.txt,SA01
3,-0.039062,-0.703125,-0.300781,-6.347656,-39.489746,-13.854980,-0.033203,-0.680664,-0.170898,adl,D01_SA01_R01.txt,SA01
4,-0.082031,-0.746094,-0.246094,-7.812500,-41.198730,-11.657715,-0.072266,-0.723633,-0.129883,adl,D01_SA01_R01.txt,SA01


In [6]:
def segment_data(df, window_size, overlap):
    # Calculate the step size based on the window size and overlap
    step = int(window_size * (1 - overlap))

    # Obtain the sensor columns
    sensor_cols = df.columns[:-3]  # Exclude label, filename, subject

    features = []
    labels = []

    for filename, group in df.groupby('filename'):
        group = group.reset_index(drop=True)
        for start in range(0, len(group) - window_size + 1, step):
            window = group.iloc[start:start + window_size]

            # Extract features and labels
            feature = window[sensor_cols].values
            label = window['label'].value_counts().idxmax()  # Most frequent label in the window

            features.append(feature)
            labels.append(label)

    return np.array(features), np.array(labels)

In [7]:
def preprocess_for_deep_learning(df, window_size=800, overlap=0.5):
    # Extract unique subjects
    subjects = df['subject'].unique()

    # Subject-wise split
    train_subjects, test_subjects = train_test_split(subjects, test_size=0.2, random_state=42)

    # Assign samples based on subject
    train_data = df[df['subject'].isin(train_subjects)]
    test_data = df[df['subject'].isin(test_subjects)]

    # Segment data into windows
    X_train, y_train = segment_data(train_data, window_size, overlap)
    X_test, y_test = segment_data(test_data, window_size, overlap)

    # Normalize using training statistics
    mean = X_train.mean(axis=(0, 1), keepdims=True)
    std = X_train.std(axis=(0, 1), keepdims=True)
    X_train = (X_train - mean) / std
    X_test = (X_test - mean) / std

    # Map labels to numeric (binary classification)
    label_map = {'adl': 0, 'fall': 1}
    y_train = np.vectorize(label_map.get)(y_train)
    y_test = np.vectorize(label_map.get)(y_test)

    return X_train, X_test, y_train, y_test

In [8]:
# Preprocess the data
X_train, X_test, y_train, y_test = preprocess_for_deep_learning(sisfall_data)

In [9]:
# Check the shape
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((10001, 800, 9), (10001,), (2482, 800, 9), (2482,))

## Training Procedure

Models were trained using mini-batch gradient descent, with a batch size 64, over 50 epochs. Early stopping was employed to prevent overfitting, stopping the training process when the validation loss ceased to improve after a defined number of epochs. Additionally, class weights were computed to address class imbalance, ensuring the model effectively learned from both fall and non-fall classes.

In [10]:
EPOCHS = 50
BATCH_SIZE = 64
N_TRIALS = 10

## Evaluation Metrics

Model performance was assessed using multiple metrics, including accuracy, precision, recall, F1-score, and ROC-AUC. These metrics comprehensively evaluate the model's ability to identify fall incidents while minimizing false alarms correctly, a critical balance in fall detection applications.

## Convolutional Neural Network (CNN)

The CNN architecture leverages convolutional layers to automatically extract temporal features from sensor data, effectively identifying local patterns characteristic of fall events.
Convolutional Layers: Multiple 1D convolutional layers with optimized filters and kernel sizes were applied, extracting local temporal patterns.

- MaxPooling Layers: Utilized to reduce dimensionality and computational load.
- Fully Connected Layers: Dense layers processed the extracted features to provide a high-level abstraction.
- Output Layer: Sigmoid activation was used to perform binary classification.

Hyperparameters (e.g., filters, kernel size, dropout, learning rate) were optimized via Optuna.

In [11]:
def build_cnn_model(input_shape, num_classes=1, hp=None):
    model = models.Sequential()
    model.add(layers.Input(shape=input_shape))

    model.add(layers.Conv1D(filters=hp['filters1'], kernel_size=hp['kernel1'], activation='relu'))
    model.add(layers.MaxPooling1D(pool_size=2))

    if hp['use_second_conv']:
        model.add(layers.Conv1D(filters=hp['filters2'], kernel_size=hp['kernel2'], activation='relu'))
        model.add(layers.MaxPooling1D(pool_size=2))

    model.add(layers.GlobalAveragePooling1D())

    model.add(layers.Dense(hp['dense_units'], activation='relu'))
    model.add(layers.Dropout(hp['dropout']))
    model.add(layers.Dense(num_classes, activation='sigmoid'))

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp['lr']),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

In [12]:
def objective(trial):
    hp = {
        'filters1': trial.suggest_categorical('filters1', [16, 32, 64]),
        'kernel1': trial.suggest_categorical('kernel1', [3, 5, 7]),
        'use_second_conv': trial.suggest_categorical('use_second_conv', [True, False]),
        'filters2': trial.suggest_categorical('filters2', [32, 64, 128]),
        'kernel2': trial.suggest_categorical('kernel2', [3, 5]),
        'dense_units': trial.suggest_int('dense_units', 32, 128),
        'dropout': trial.suggest_float('dropout', 0.2, 0.5),
        'lr': trial.suggest_float('lr', 1e-4, 1e-2, log=True)
    }

    skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
    scores = []
    for train_idx, val_idx in skf.split(X_train, y_train):
        X_tr, X_val = X_train[train_idx], X_train[val_idx]
        y_tr, y_val = y_train[train_idx], y_train[val_idx]

        class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_tr), y=y_tr)
        class_weight_dict = dict(enumerate(class_weights))

        model = build_cnn_model(X_tr.shape[1:], num_classes=1, hp=hp)

        early_stop = callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

        model.fit(X_tr, y_tr, epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=1,
                  class_weight=class_weight_dict, validation_data=(X_val, y_val),
                  callbacks=[early_stop])

        y_val_pred = (model.predict(X_val).flatten() > 0.5).astype(int)
        scores.append(f1_score(y_val, y_val_pred))

    return np.mean(scores)

In [13]:
# Run Optuna hyperparameter tuning
cnn_study = optuna.create_study(direction='maximize')
cnn_study.optimize(objective, n_trials=N_TRIALS)

[I 2025-05-06 23:32:00,204] A new study created in memory with name: no-name-0bad76ab-ed02-4655-b33c-cc14b72fc11b
I0000 00:00:1746570721.878318  106205 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4057 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2060 with Max-Q Design, pci bus id: 0000:01:00.0, compute capability: 7.5


Epoch 1/50


I0000 00:00:1746570724.892268  106488 service.cc:152] XLA service 0x7fd6e8023900 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1746570724.892345  106488 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 2060 with Max-Q Design, Compute Capability 7.5
2025-05-06 23:32:04.959172: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1746570725.232357  106488 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1746570727.175704  106488 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


105/105 ━━━━━━━━━━━━━━━━━━━━ 9s 43ms/step - accuracy: 0.8079 - loss: 0.4621 - val_accuracy: 0.8443 - val_loss: 0.3695
Epoch 2/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.8450 - loss: 0.3590 - val_accuracy: 0.8488 - val_loss: 0.3304
Epoch 3/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.8510 - loss: 0.3204 - val_accuracy: 0.8626 - val_loss: 0.3198
Epoch 4/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.8562 - loss: 0.3184 - val_accuracy: 0.8710 - val_loss: 0.2941
Epoch 5/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.8618 - loss: 0.3102 - val_accuracy: 0.8716 - val_loss: 0.2892
Epoch 6/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.8648 - loss: 0.2969 - val_accuracy: 0.8752 - val_loss: 0.2774
Epoch 7/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.8550 - loss: 0.3003 - val_accuracy: 0.8767 - val_loss: 0.2838
Epoch 8/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.8762 - loss: 0.2696 - val_accuracy: 0.874

[I 2025-05-06 23:34:44,338] Trial 0 finished with value: 0.9171968997689518 and parameters: {'filters1': 64, 'kernel1': 3, 'use_second_conv': True, 'filters2': 128, 'kernel2': 5, 'dense_units': 50, 'dropout': 0.3888348036729762, 'lr': 0.001192265824418235}. Best is trial 0 with value: 0.9171968997689518.


...

[I 2025-05-06 23:49:13,532] Trial 7 finished with value: 0.8510128795923199 and parameters: {'filters1': 16, 'kernel1': 7, 'use_second_conv': False, 'filters2': 128, 'kernel2': 5, 'dense_units': 38, 'dropout': 0.21172642686938536, 'lr': 0.009179706586236607}. Best is trial 0 with value: 0.9171968997689518.


Epoch 1/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 7s 38ms/step - accuracy: 0.7698 - loss: 0.5298 - val_accuracy: 0.8374 - val_loss: 0.3932
Epoch 2/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.8347 - loss: 0.3925 - val_accuracy: 0.8437 - val_loss: 0.3580
Epoch 3/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.8462 - loss: 0.3570 - val_accuracy: 0.8527 - val_loss: 0.3375
Epoch 4/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.8426 - loss: 0.3448 - val_accuracy: 0.8542 - val_loss: 0.3222
Epoch 5/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.8515 - loss: 0.3294 - val_accuracy: 0.8548 - val_loss: 0.3121
Epoch 6/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.8597 - loss: 0.3206 - val_accuracy: 0.8560 - val_loss: 0.3163
Epoch 7/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.8522 - loss: 0.3178 - val_accuracy: 0.8677 - val_loss: 0.2979
Epoch 8/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.8592 - loss: 0.3003 - val_accu

[I 2025-05-06 23:51:56,542] Trial 8 finished with value: 0.8771150880865504 and parameters: {'filters1': 64, 'kernel1': 5, 'use_second_conv': True, 'filters2': 32, 'kernel2': 3, 'dense_units': 70, 'dropout': 0.20449790831966355, 'lr': 0.0006457064513729101}. Best is trial 0 with value: 0.9171968997689518.


Epoch 1/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 8s 47ms/step - accuracy: 0.7932 - loss: 0.4520 - val_accuracy: 0.8464 - val_loss: 0.3383
Epoch 2/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.8440 - loss: 0.3438 - val_accuracy: 0.8551 - val_loss: 0.3225
Epoch 3/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.8507 - loss: 0.3183 - val_accuracy: 0.8653 - val_loss: 0.3030
Epoch 4/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.8597 - loss: 0.3164 - val_accuracy: 0.8668 - val_loss: 0.3005
Epoch 5/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.8716 - loss: 0.2855 - val_accuracy: 0.8731 - val_loss: 0.2827
Epoch 6/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.8743 - loss: 0.2768 - val_accuracy: 0.8812 - val_loss: 0.2639
Epoch 7/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.8865 - loss: 0.2588 - val_accuracy: 0.9010 - val_loss: 0.2538
Epoch 8/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.8994 - loss: 0.2368 - val_accu

[I 2025-05-06 23:54:08,700] Trial 9 finished with value: 0.9252124036185673 and parameters: {'filters1': 32, 'kernel1': 7, 'use_second_conv': True, 'filters2': 128, 'kernel2': 5, 'dense_units': 64, 'dropout': 0.328632646675748, 'lr': 0.005009169687821036}. Best is trial 9 with value: 0.9252124036185673.


In [14]:
# Get the best hyperparameters
best_params = cnn_study.best_params
print("Best CNN hyperparameters:", best_params)

Best CNN hyperparameters: {'filters1': 32, 'kernel1': 7, 'use_second_conv': True, 'filters2': 128, 'kernel2': 5, 'dense_units': 64, 'dropout': 0.328632646675748, 'lr': 0.005009169687821036}


In [15]:
cnn_model = build_cnn_model(X_train.shape[1:], num_classes=1, hp=best_params)
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weight_dict = dict(enumerate(class_weights))

early_stop_final = callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
cnn_model.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=1,
              class_weight=class_weight_dict, validation_split=0.2,
              callbacks=[early_stop_final])

Epoch 1/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 5s 22ms/step - accuracy: 0.8351 - loss: 0.4265 - val_accuracy: 0.7306 - val_loss: 0.6204
Epoch 2/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.8618 - loss: 0.3332 - val_accuracy: 0.7086 - val_loss: 0.9178
Epoch 3/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.8680 - loss: 0.3199 - val_accuracy: 0.7581 - val_loss: 0.5023
Epoch 4/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - accuracy: 0.8798 - loss: 0.2783 - val_accuracy: 0.7816 - val_loss: 0.5089
Epoch 5/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - accuracy: 0.8863 - loss: 0.2740 - val_accuracy: 0.7616 - val_loss: 0.6552
Epoch 6/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - accuracy: 0.8986 - loss: 0.2437 - val_accuracy: 0.7351 - val_loss: 0.6744
Epoch 7/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.9025 - loss: 0.2268 - val_accuracy: 0.7671 - val_loss: 0.6041
Epoch 8/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - accuracy: 0.9256 - loss: 0.1908 - val_accu

In [16]:
# Evaluate on test set
y_pred_prob = cnn_model.predict(X_test).flatten()
y_pred = (y_pred_prob > 0.5).astype(int)

78/78 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step


In [17]:
# Metrics
test_accuracy = accuracy_score(y_test, y_pred)
test_precision = precision_score(y_test, y_pred)
test_recall = recall_score(y_test, y_pred)
test_f1 = f1_score(y_test, y_pred)
test_auc = roc_auc_score(y_test, y_pred_prob)

print("Test Accuracy:", test_accuracy)
print("Test Precision:", test_precision)
print("Test Recall:", test_recall)
print("Test F1-score:", test_f1)
print("Test AUC:", test_auc)

Test Accuracy: 0.8182917002417406
Test Precision: 0.7609958506224066
Test Recall: 0.8490740740740741
Test F1-score: 0.8026258205689278
Test AUC: 0.8917914883499762


In [18]:
# Save the model
cnn_model.save('cnn_model.keras')

## Long Short-Term Memory (LSTM)

An LSTM-based model was implemented to explicitly capture temporal dependencies in sensor data due to its capability of managing long-range sequences.
LSTM Layers: Multiple stacked LSTM layers were configured to learn complex sequential patterns, handling potential vanishing gradient problems inherent in traditional RNNs.

- Dropout Layers: Regularized intermediate layers to reduce overfitting.
- Dense Layers: Integrated to interpret the temporal features extracted by the LSTM units.
- Output Layer: A sigmoid activation function provided binary predictions distinguishing falls from non-falls.

Hyperparameter optimization was similarly conducted to tune hidden units, learning rate, dropout rates, and batch size.

In [20]:
def build_lstm_model(input_shape, num_classes=1, hp=None):
    model = models.Sequential()
    model.add(layers.Input(shape=input_shape))

    model.add(layers.LSTM(units=hp['lstm_units'], return_sequences=hp['return_sequences']))
    if hp['return_sequences']:
        model.add(layers.LSTM(units=hp['lstm_units2']))

    model.add(layers.Dropout(hp['dropout']))
    model.add(layers.Dense(hp['dense_units'], activation='relu'))
    model.add(layers.Dense(num_classes, activation='sigmoid'))

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp['lr']),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

In [21]:
def objective_lstm(trial):
    hp = {
        'lstm_units': trial.suggest_int('lstm_units', 32, 128),
        'return_sequences': trial.suggest_categorical('return_sequences', [True, False]),
        'lstm_units2': trial.suggest_int('lstm_units2', 32, 128),
        'dense_units': trial.suggest_int('dense_units', 32, 128),
        'dropout': trial.suggest_float('dropout', 0.2, 0.5),
        'lr': trial.suggest_float('lr', 1e-4, 1e-2, log=True)
    }

    skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
    scores = []
    for train_idx, val_idx in skf.split(X_train, y_train):
        X_tr, X_val = X_train[train_idx], X_train[val_idx]
        y_tr, y_val = y_train[train_idx], y_train[val_idx]

        class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_tr), y=y_tr)
        class_weight_dict = dict(enumerate(class_weights))

        model = build_lstm_model(X_tr.shape[1:], num_classes=1, hp=hp)

        early_stop = callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

        model.fit(X_tr, y_tr, epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=1,
                  class_weight=class_weight_dict, validation_data=(X_val, y_val),
                  callbacks=[early_stop])

        y_val_pred = (model.predict(X_val).flatten() > 0.5).astype(int)
        scores.append(f1_score(y_val, y_val_pred))

    return np.mean(scores)

In [22]:
lstm_study = optuna.create_study(direction='maximize')
lstm_study.optimize(objective_lstm, n_trials=N_TRIALS)

[I 2025-05-06 23:54:41,450] A new study created in memory with name: no-name-e96f3795-8b1d-4342-83dd-f814be23d0f2


Epoch 1/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 32s 255ms/step - accuracy: 0.7739 - loss: 0.5076 - val_accuracy: 0.8578 - val_loss: 0.3728
Epoch 2/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 24s 233ms/step - accuracy: 0.8662 - loss: 0.3544 - val_accuracy: 0.8611 - val_loss: 0.3532
Epoch 3/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 26s 246ms/step - accuracy: 0.8720 - loss: 0.3414 - val_accuracy: 0.8602 - val_loss: 0.3472
Epoch 4/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 23s 224ms/step - accuracy: 0.8723 - loss: 0.3280 - val_accuracy: 0.8617 - val_loss: 0.3443
Epoch 5/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 25s 240ms/step - accuracy: 0.8724 - loss: 0.3324 - val_accuracy: 0.8617 - val_loss: 0.3448
Epoch 6/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 26s 244ms/step - accuracy: 0.8758 - loss: 0.3128 - val_accuracy: 0.8644 - val_loss: 0.3421
Epoch 7/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 25s 240ms/step - accuracy: 0.8722 - loss: 0.3243 - val_accuracy: 0.8647 - val_loss: 0.3293
Epoch 8/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 25s 239ms/step - accuracy: 0.8767 - loss: 0

[I 2025-05-07 00:07:55,402] Trial 0 finished with value: 0.8382096013485124 and parameters: {'lstm_units': 82, 'return_sequences': True, 'lstm_units2': 119, 'dense_units': 68, 'dropout': 0.20246942237568494, 'lr': 0.0005687371450135624}. Best is trial 0 with value: 0.8382096013485124.


Epoch 1/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 24s 214ms/step - accuracy: 0.7835 - loss: 0.4936 - val_accuracy: 0.8530 - val_loss: 0.3880
Epoch 2/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 24s 232ms/step - accuracy: 0.8616 - loss: 0.3802 - val_accuracy: 0.8584 - val_loss: 0.3591
Epoch 3/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 21s 198ms/step - accuracy: 0.8585 - loss: 0.3625 - val_accuracy: 0.8584 - val_loss: 0.4284
Epoch 4/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 21s 203ms/step - accuracy: 0.8633 - loss: 0.3637 - val_accuracy: 0.8638 - val_loss: 0.3421
Epoch 5/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 22s 206ms/step - accuracy: 0.8682 - loss: 0.3358 - val_accuracy: 0.8518 - val_loss: 0.3472
Epoch 6/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 21s 203ms/step - accuracy: 0.8683 - loss: 0.3328 - val_accuracy: 0.8560 - val_loss: 0.3326
Epoch 7/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 22s 207ms/step - accuracy: 0.8705 - loss: 0.3175 - val_accuracy: 0.8599 - val_loss: 0.3388
Epoch 8/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 21s 204ms/step - accuracy: 0.8561 - loss: 0

[I 2025-05-07 00:17:21,678] Trial 1 finished with value: 0.8283867714169365 and parameters: {'lstm_units': 90, 'return_sequences': True, 'lstm_units2': 66, 'dense_units': 75, 'dropout': 0.40939912504018383, 'lr': 0.0018329173041901556}. Best is trial 0 with value: 0.8382096013485124.


...

[I 2025-05-07 01:15:58,515] Trial 7 finished with value: 0.8421401819928976 and parameters: {'lstm_units': 96, 'return_sequences': False, 'lstm_units2': 40, 'dense_units': 45, 'dropout': 0.26808081063802525, 'lr': 0.0008202177886796195}. Best is trial 3 with value: 0.8424065252977876.


Epoch 1/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 23s 205ms/step - accuracy: 0.8182 - loss: 0.4846 - val_accuracy: 0.8557 - val_loss: 0.3723
Epoch 2/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 22s 206ms/step - accuracy: 0.8513 - loss: 0.3845 - val_accuracy: 0.8626 - val_loss: 0.3659
Epoch 3/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 22s 210ms/step - accuracy: 0.8596 - loss: 0.3851 - val_accuracy: 0.8500 - val_loss: 0.3949
Epoch 4/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 22s 209ms/step - accuracy: 0.8457 - loss: 0.4047 - val_accuracy: 0.8506 - val_loss: 0.3975
Epoch 5/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 21s 204ms/step - accuracy: 0.8538 - loss: 0.3834 - val_accuracy: 0.8092 - val_loss: 0.4338
105/105 ━━━━━━━━━━━━━━━━━━━━ 7s 69ms/step
Epoch 1/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 23s 209ms/step - accuracy: 0.8019 - loss: 0.4848 - val_accuracy: 0.8509 - val_loss: 0.3982
Epoch 2/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 22s 206ms/step - accuracy: 0.8558 - loss: 0.4049 - val_accuracy: 0.8602 - val_loss: 0.3987
Epoch 3/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 2

[I 2025-05-07 01:25:41,564] Trial 8 finished with value: 0.833271083383497 and parameters: {'lstm_units': 72, 'return_sequences': True, 'lstm_units2': 53, 'dense_units': 71, 'dropout': 0.23827751443861414, 'lr': 0.003479740652957881}. Best is trial 3 with value: 0.8424065252977876.


Epoch 1/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 14s 121ms/step - accuracy: 0.6338 - loss: 0.6059 - val_accuracy: 0.7639 - val_loss: 0.4980
Epoch 2/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 12s 118ms/step - accuracy: 0.7797 - loss: 0.4711 - val_accuracy: 0.8425 - val_loss: 0.4157
Epoch 3/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 12s 117ms/step - accuracy: 0.8543 - loss: 0.4091 - val_accuracy: 0.8503 - val_loss: 0.3977
Epoch 4/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 12s 114ms/step - accuracy: 0.8441 - loss: 0.4097 - val_accuracy: 0.8536 - val_loss: 0.3807
Epoch 5/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 12s 117ms/step - accuracy: 0.8599 - loss: 0.3793 - val_accuracy: 0.8581 - val_loss: 0.3705
Epoch 6/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 12s 114ms/step - accuracy: 0.8615 - loss: 0.3701 - val_accuracy: 0.8572 - val_loss: 0.3644
Epoch 7/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 12s 112ms/step - accuracy: 0.8619 - loss: 0.3572 - val_accuracy: 0.8608 - val_loss: 0.3703
Epoch 8/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 12s 118ms/step - accuracy: 0.8389 - loss: 0

[I 2025-05-07 01:37:44,668] Trial 9 finished with value: 0.8410116746807282 and parameters: {'lstm_units': 66, 'return_sequences': False, 'lstm_units2': 44, 'dense_units': 80, 'dropout': 0.2860453080261605, 'lr': 0.00018080212779777506}. Best is trial 3 with value: 0.8424065252977876.


In [23]:
# Get the best hyperparameters
best_lstm_params = lstm_study.best_params
print("Best LSTM hyperparameters:", best_lstm_params)

Best LSTM hyperparameters: {'lstm_units': 100, 'return_sequences': True, 'lstm_units2': 124, 'dense_units': 76, 'dropout': 0.46646724486425, 'lr': 0.00013450442117152523}


In [24]:
lstm_model = build_lstm_model(X_train.shape[1:], num_classes=1, hp=best_lstm_params)
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weight_dict = dict(enumerate(class_weights))

early_stop_final = callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
lstm_model.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=1,
               class_weight=class_weight_dict, validation_split=0.2,
               callbacks=[early_stop_final])

Epoch 1/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 28s 212ms/step - accuracy: 0.6509 - loss: 0.5820 - val_accuracy: 0.6897 - val_loss: 0.5995
Epoch 2/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 27s 212ms/step - accuracy: 0.8660 - loss: 0.3972 - val_accuracy: 0.7431 - val_loss: 0.5190
Epoch 3/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 27s 217ms/step - accuracy: 0.8703 - loss: 0.3773 - val_accuracy: 0.7771 - val_loss: 0.4698
Epoch 4/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 27s 217ms/step - accuracy: 0.8773 - loss: 0.3515 - val_accuracy: 0.7926 - val_loss: 0.4500
Epoch 5/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 28s 222ms/step - accuracy: 0.8726 - loss: 0.3379 - val_accuracy: 0.7891 - val_loss: 0.4524
Epoch 6/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 27s 217ms/step - accuracy: 0.8749 - loss: 0.3276 - val_accuracy: 0.8161 - val_loss: 0.3672
Epoch 7/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 27s 216ms/step - accuracy: 0.8775 - loss: 0.3204 - val_accuracy: 0.8026 - val_loss: 0.4160
Epoch 8/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 29s 234ms/step - accuracy: 0.8838 - loss: 0

In [25]:
y_pred_prob = lstm_model.predict(X_test).flatten()
y_pred = (y_pred_prob > 0.5).astype(int)

78/78 ━━━━━━━━━━━━━━━━━━━━ 6s 76ms/step


In [26]:
test_accuracy = accuracy_score(y_test, y_pred)
test_precision = precision_score(y_test, y_pred)
test_recall = recall_score(y_test, y_pred)
test_f1 = f1_score(y_test, y_pred)
test_auc = roc_auc_score(y_test, y_pred_prob)

print("LSTM Test Accuracy:", test_accuracy)
print("LSTM Test Precision:", test_precision)
print("LSTM Test Recall:", test_recall)
print("LSTM Test F1-score:", test_f1)
print("LSTM Test AUC:", test_auc)

LSTM Test Accuracy: 0.8642224012892828
LSTM Test Precision: 0.8667324777887463
LSTM Test Recall: 0.812962962962963
LSTM Test F1-score: 0.8389870998566651
LSTM Test AUC: 0.9173693665134465


In [27]:
# Save the model
lstm_model.save('lstm_model.keras')

## Hybrid Model (CNN-LSTM)

The hybrid CNN-LSTM architecture integrates the strengths of convolutional and recurrent layers. CNN layers initially extract meaningful spatial-temporal features, while subsequent LSTM layers model sequential dependencies from these features.

- Convolutional Layers: Extract initial local patterns and reduce sequence complexity.
- LSTM Layers: Analyze extracted features to capture temporal context and dependencies effectively.
- Fully Connected Layers: Provide an abstraction of temporal patterns.
- Output Layer: Binary output indicating fall or non-fall events.

Optuna optimized both CNN and LSTM hyperparameters to ensure balanced performance of both model components.

In [29]:
def build_cnn_lstm_model(input_shape, num_classes=1, hp=None):
    model = models.Sequential()
    model.add(layers.Input(shape=input_shape))

    model.add(layers.Conv1D(filters=hp['filters'], kernel_size=hp['kernel_size'], activation='relu'))
    model.add(layers.MaxPooling1D(pool_size=2))
    model.add(layers.LSTM(units=hp['lstm_units']))

    model.add(layers.Dropout(hp['dropout']))
    model.add(layers.Dense(hp['dense_units'], activation='relu'))
    model.add(layers.Dense(num_classes, activation='sigmoid'))

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp['lr']),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

In [30]:
def objective_cnn_lstm(trial):
    hp = {
        'filters': trial.suggest_categorical('filters', [16, 32, 64]),
        'kernel_size': trial.suggest_categorical('kernel_size', [3, 5, 7]),
        'lstm_units': trial.suggest_int('lstm_units', 32, 128),
        'dense_units': trial.suggest_int('dense_units', 32, 128),
        'dropout': trial.suggest_float('dropout', 0.2, 0.5),
        'lr': trial.suggest_float('lr', 1e-4, 1e-2, log=True)
    }

    skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
    scores = []
    for train_idx, val_idx in skf.split(X_train, y_train):
        X_tr, X_val = X_train[train_idx], X_train[val_idx]
        y_tr, y_val = y_train[train_idx], y_train[val_idx]

        class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_tr), y=y_tr)
        class_weight_dict = dict(enumerate(class_weights))

        model = build_cnn_lstm_model(X_tr.shape[1:], num_classes=1, hp=hp)

        early_stop = callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

        model.fit(X_tr, y_tr, epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=1,
                  class_weight=class_weight_dict, validation_data=(X_val, y_val),
                  callbacks=[early_stop])

        y_val_pred = (model.predict(X_val).flatten() > 0.5).astype(int)
        scores.append(f1_score(y_val, y_val_pred))

    return np.mean(scores)

In [31]:
cnn_lstm_study = optuna.create_study(direction='maximize')
cnn_lstm_study.optimize(objective_cnn_lstm, n_trials=N_TRIALS)

[I 2025-05-07 01:42:55,418] A new study created in memory with name: no-name-4f8dba39-029f-4421-ab71-10c994c32454


Epoch 1/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 10s 79ms/step - accuracy: 0.8036 - loss: 0.4640 - val_accuracy: 0.8554 - val_loss: 0.3673
Epoch 2/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 8s 72ms/step - accuracy: 0.8697 - loss: 0.3509 - val_accuracy: 0.8623 - val_loss: 0.3390
Epoch 3/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 8s 73ms/step - accuracy: 0.8666 - loss: 0.3558 - val_accuracy: 0.8641 - val_loss: 0.3307
Epoch 4/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 7s 71ms/step - accuracy: 0.8627 - loss: 0.3418 - val_accuracy: 0.8614 - val_loss: 0.3534
Epoch 5/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 7s 65ms/step - accuracy: 0.8641 - loss: 0.3590 - val_accuracy: 0.8611 - val_loss: 0.3600
Epoch 6/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 7s 67ms/step - accuracy: 0.8615 - loss: 0.3433 - val_accuracy: 0.8677 - val_loss: 0.3204
Epoch 7/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 6s 56ms/step - accuracy: 0.8618 - loss: 0.3241 - val_accuracy: 0.8608 - val_loss: 0.3335
Epoch 8/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 5s 49ms/step - accuracy: 0.8724 - loss: 0.3056 - val_acc

[I 2025-05-07 01:46:23,315] Trial 0 finished with value: 0.845877773253021 and parameters: {'filters': 16, 'kernel_size': 5, 'lstm_units': 37, 'dense_units': 81, 'dropout': 0.22380543227925997, 'lr': 0.0070942919009696825}. Best is trial 0 with value: 0.845877773253021.


Epoch 1/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 9s 70ms/step - accuracy: 0.7240 - loss: 0.5526 - val_accuracy: 0.8437 - val_loss: 0.3898
Epoch 2/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 7s 68ms/step - accuracy: 0.8597 - loss: 0.3869 - val_accuracy: 0.8575 - val_loss: 0.3591
Epoch 3/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 7s 67ms/step - accuracy: 0.8713 - loss: 0.3511 - val_accuracy: 0.8572 - val_loss: 0.3483
Epoch 4/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 7s 65ms/step - accuracy: 0.8702 - loss: 0.3385 - val_accuracy: 0.8560 - val_loss: 0.3404
Epoch 5/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 7s 65ms/step - accuracy: 0.8614 - loss: 0.3311 - val_accuracy: 0.8587 - val_loss: 0.3270
Epoch 6/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 7s 68ms/step - accuracy: 0.8708 - loss: 0.3248 - val_accuracy: 0.8431 - val_loss: 0.3959
Epoch 7/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 7s 65ms/step - accuracy: 0.8685 - loss: 0.3491 - val_accuracy: 0.8605 - val_loss: 0.3303
Epoch 8/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 7s 69ms/step - accuracy: 0.8724 - loss: 0.3184 - val_accu

[I 2025-05-07 01:49:57,429] Trial 1 finished with value: 0.8409046255524046 and parameters: {'filters': 32, 'kernel_size': 7, 'lstm_units': 49, 'dense_units': 91, 'dropout': 0.4238483799261735, 'lr': 0.0003649540923263948}. Best is trial 0 with value: 0.845877773253021.


...

[I 2025-05-07 02:24:38,512] Trial 7 finished with value: 0.8640423411969476 and parameters: {'filters': 32, 'kernel_size': 5, 'lstm_units': 84, 'dense_units': 53, 'dropout': 0.36166914043585596, 'lr': 0.0098836461777134}. Best is trial 7 with value: 0.8640423411969476.


Epoch 1/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 10s 85ms/step - accuracy: 0.7533 - loss: 0.5233 - val_accuracy: 0.8614 - val_loss: 0.3607
Epoch 2/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 8s 74ms/step - accuracy: 0.8689 - loss: 0.3535 - val_accuracy: 0.8584 - val_loss: 0.3487
Epoch 3/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 8s 78ms/step - accuracy: 0.8713 - loss: 0.3321 - val_accuracy: 0.8668 - val_loss: 0.3344
Epoch 4/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 8s 78ms/step - accuracy: 0.8641 - loss: 0.3343 - val_accuracy: 0.8569 - val_loss: 0.3539
Epoch 5/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 8s 77ms/step - accuracy: 0.8833 - loss: 0.3072 - val_accuracy: 0.8566 - val_loss: 0.3306
Epoch 6/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 8s 76ms/step - accuracy: 0.8751 - loss: 0.2925 - val_accuracy: 0.8644 - val_loss: 0.3194
Epoch 7/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 8s 77ms/step - accuracy: 0.8782 - loss: 0.2856 - val_accuracy: 0.8650 - val_loss: 0.3260
Epoch 8/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 8s 77ms/step - accuracy: 0.8806 - loss: 0.2822 - val_acc

[I 2025-05-07 02:30:25,468] Trial 8 finished with value: 0.855502251705914 and parameters: {'filters': 64, 'kernel_size': 7, 'lstm_units': 115, 'dense_units': 111, 'dropout': 0.2131931885441707, 'lr': 0.00029121425222516066}. Best is trial 7 with value: 0.8640423411969476.


Epoch 1/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 11s 90ms/step - accuracy: 0.7501 - loss: 0.5957 - val_accuracy: 0.8500 - val_loss: 0.3852
Epoch 2/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 8s 78ms/step - accuracy: 0.8566 - loss: 0.3791 - val_accuracy: 0.8536 - val_loss: 0.3641
Epoch 3/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 9s 85ms/step - accuracy: 0.8698 - loss: 0.3463 - val_accuracy: 0.8281 - val_loss: 0.3761
Epoch 4/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 8s 81ms/step - accuracy: 0.8674 - loss: 0.3314 - val_accuracy: 0.8569 - val_loss: 0.3452
Epoch 5/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 9s 82ms/step - accuracy: 0.8662 - loss: 0.3299 - val_accuracy: 0.8575 - val_loss: 0.3289
Epoch 6/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 8s 80ms/step - accuracy: 0.8631 - loss: 0.3256 - val_accuracy: 0.8608 - val_loss: 0.3255
Epoch 7/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 9s 82ms/step - accuracy: 0.8680 - loss: 0.3060 - val_accuracy: 0.8659 - val_loss: 0.3432
Epoch 8/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 9s 81ms/step - accuracy: 0.8746 - loss: 0.3095 - val_acc

[I 2025-05-07 02:35:20,647] Trial 9 finished with value: 0.8424775866563273 and parameters: {'filters': 64, 'kernel_size': 7, 'lstm_units': 121, 'dense_units': 47, 'dropout': 0.27062737415951293, 'lr': 0.00011029772332526185}. Best is trial 7 with value: 0.8640423411969476.


In [32]:
# Get the best hyperparameters
best_cnn_lstm_params = cnn_lstm_study.best_params
print("Best CNN-LSTM hyperparameters:", best_cnn_lstm_params)

Best CNN-LSTM hyperparameters: {'filters': 32, 'kernel_size': 5, 'lstm_units': 84, 'dense_units': 53, 'dropout': 0.36166914043585596, 'lr': 0.0098836461777134}


In [33]:
cnn_lstm_model = build_cnn_lstm_model(X_train.shape[1:], num_classes=1, hp=best_cnn_lstm_params)
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weight_dict = dict(enumerate(class_weights))

early_stop_final = callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
cnn_lstm_model.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=1,
                   class_weight=class_weight_dict, validation_split=0.2,
                   callbacks=[early_stop_final])

Epoch 1/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 9s 61ms/step - accuracy: 0.8497 - loss: 0.4216 - val_accuracy: 0.7971 - val_loss: 0.3690
Epoch 2/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 7s 59ms/step - accuracy: 0.8690 - loss: 0.3607 - val_accuracy: 0.7671 - val_loss: 0.5530
Epoch 3/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 7s 59ms/step - accuracy: 0.8679 - loss: 0.3721 - val_accuracy: 0.6237 - val_loss: 0.7617
Epoch 4/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 7s 57ms/step - accuracy: 0.8614 - loss: 0.3751 - val_accuracy: 0.7501 - val_loss: 0.4639
Epoch 5/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 7s 59ms/step - accuracy: 0.8728 - loss: 0.3412 - val_accuracy: 0.7561 - val_loss: 0.5696
Epoch 6/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 7s 60ms/step - accuracy: 0.8686 - loss: 0.3290 - val_accuracy: 0.7506 - val_loss: 0.6619


In [34]:
y_pred_prob = cnn_lstm_model.predict(X_test).flatten()
y_pred = (y_pred_prob > 0.5).astype(int)

78/78 ━━━━━━━━━━━━━━━━━━━━ 2s 27ms/step


In [35]:
test_accuracy = accuracy_score(y_test, y_pred)
test_precision = precision_score(y_test, y_pred)
test_recall = recall_score(y_test, y_pred)
test_f1 = f1_score(y_test, y_pred)
test_auc = roc_auc_score(y_test, y_pred_prob)

print("CNN-LSTM Test Accuracy:", test_accuracy)
print("CNN-LSTM Test Precision:", test_precision)
print("CNN-LSTM Test Recall:", test_recall)
print("CNN-LSTM Test F1-score:", test_f1)
print("CNN-LSTM Test AUC:", test_auc)

CNN-LSTM Test Accuracy: 0.8565672844480258
CNN-LSTM Test Precision: 0.8851063829787233
CNN-LSTM Test Recall: 0.7703703703703704
CNN-LSTM Test F1-score: 0.8237623762376237
CNN-LSTM Test AUC: 0.9128572938130712


In [36]:
# Save the model
cnn_lstm_model.save('cnn_lstm_model.keras')

## Transformer Model

The Transformer architecture was selected due to its effectiveness in modeling sequential data through self-attention mechanisms. Transformers allow simultaneous attention to all positions in the sequence, effectively capturing long-range dependencies and intricate temporal interactions.

- Positional Encoding: Embedded temporal positional information enabling the model to distinguish relative positioning within the sequence.
- Multi-head Self-Attention Layers: Allowed the model to focus simultaneously on multiple temporal patterns and interactions within the sensor data, capturing nuanced indicators of fall events
- Feed-Forward Neural Network Layers: Provided a means for deeper abstraction and interpretation of temporal relationships.
- Layer Normalization: Enhanced model stability during training.
- Output Layer: Sigmoid activation function performed binary classification tasks.

Hyperparameter tuning via Optuna optimized attention heads, embedding dimensions, learning rates, dropout rates, and other critical parameters, ensuring that the Transformer leveraged its attention capabilities efficiently for fall detection tasks.

In [11]:
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout):
    # Self-attention block
    attention_output = layers.MultiHeadAttention(
        key_dim=head_size,
        num_heads=num_heads,
        dropout=dropout
    )(inputs, inputs)
    attention_output = layers.Dropout(dropout)(attention_output)
    attention_output = layers.Add()([inputs, attention_output])  # Residual connection
    attention_output = layers.LayerNormalization(epsilon=1e-6)(attention_output)

    # Feed-forward block
    ff_output = layers.Dense(ff_dim, activation="relu")(attention_output)
    ff_output = layers.Dropout(dropout)(ff_output)
    ff_output = layers.Dense(inputs.shape[-1])(ff_output)
    ff_output = layers.Add()([attention_output, ff_output])  # Residual connection
    ff_output = layers.LayerNormalization(epsilon=1e-6)(ff_output)

    return ff_output

In [12]:
def build_transformer_model(input_shape, num_classes=1, hp=None):
    inputs = layers.Input(shape=input_shape)
    x = inputs

    for _ in range(hp['num_transformer_blocks']):
        x = transformer_encoder(x, head_size=hp['head_size'], num_heads=hp['num_heads'], ff_dim=hp['ff_dim'],
                                dropout=hp['dropout'])

    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(hp['dense_units'], activation='relu')(x)
    x = layers.Dropout(hp['dropout'])(x)
    outputs = layers.Dense(num_classes, activation='sigmoid')(x)

    model = models.Model(inputs, outputs)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp['lr']),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

In [13]:
def objective_transformer(trial):
    hp = {
        'num_transformer_blocks': trial.suggest_int('num_transformer_blocks', 1, 3),
        'head_size': trial.suggest_categorical('head_size', [32, 64]),
        'num_heads': trial.suggest_categorical('num_heads', [2, 4]),
        'ff_dim': trial.suggest_int('ff_dim', 32, 128),
        'dense_units': trial.suggest_int('dense_units', 32, 128),
        'dropout': trial.suggest_float('dropout', 0.1, 0.5),
        'lr': trial.suggest_float('lr', 1e-4, 1e-2, log=True)
    }

    skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
    scores = []
    for train_idx, val_idx in skf.split(X_train, y_train):
        X_tr, X_val = X_train[train_idx], X_train[val_idx]
        y_tr, y_val = y_train[train_idx], y_train[val_idx]

        class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_tr), y=y_tr)
        class_weight_dict = dict(enumerate(class_weights))

        model = build_transformer_model(X_tr.shape[1:], num_classes=1, hp=hp)

        early_stop = callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

        model.fit(X_tr, y_tr, epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=1,
                  class_weight=class_weight_dict, validation_data=(X_val, y_val),
                  callbacks=[early_stop])

        y_val_pred = (model.predict(X_val).flatten() > 0.5).astype(int)
        scores.append(f1_score(y_val, y_val_pred))

    return np.mean(scores)

In [14]:
transformer_study = optuna.create_study(direction='maximize')
transformer_study.optimize(objective_transformer, n_trials=N_TRIALS)

[I 2025-05-07 13:24:42,181] A new study created in memory with name: no-name-b7a656df-3e54-4738-965c-50f3f5e8c719
I0000 00:00:1746620683.246259     978 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4057 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2060 with Max-Q Design, pci bus id: 0000:01:00.0, compute capability: 7.5


Epoch 1/50


2025-05-07 13:24:45.143808: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 192009600 exceeds 10% of free system memory.
2025-05-07 13:24:45.231293: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 192009600 exceeds 10% of free system memory.
I0000 00:00:1746620688.896950    1267 service.cc:152] XLA service 0x7f3e34007550 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1746620688.897023    1267 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 2060 with Max-Q Design, Compute Capability 7.5
2025-05-07 13:24:49.063092: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1746620689.785688    1267 cuda_dnn.cc:529] Loaded cuDNN version 90300


  1/105 ━━━━━━━━━━━━━━━━━━━━ 21:09 12s/step - accuracy: 0.6719 - loss: 0.6590

I0000 00:00:1746620697.348390    1267 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


...

105/105 ━━━━━━━━━━━━━━━━━━━━ 25s 172ms/step - accuracy: 0.7546 - loss: 0.5535 - val_accuracy: 0.8407 - val_loss: 0.4234
Epoch 2/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 11s 104ms/step - accuracy: 0.8305 - loss: 0.4425 - val_accuracy: 0.8365 - val_loss: 0.3826
Epoch 3/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 11s 104ms/step - accuracy: 0.8351 - loss: 0.4025 - val_accuracy: 0.8470 - val_loss: 0.3585
Epoch 4/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 11s 107ms/step - accuracy: 0.8447 - loss: 0.3672 - val_accuracy: 0.8575 - val_loss: 0.3447
Epoch 5/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 11s 106ms/step - accuracy: 0.8577 - loss: 0.3432 - val_accuracy: 0.8596 - val_loss: 0.3416
Epoch 6/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 12s 112ms/step - accuracy: 0.8539 - loss: 0.3389 - val_accuracy: 0.8575 - val_loss: 0.3274
Epoch 7/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 11s 109ms/step - accuracy: 0.8527 - loss: 0.3218 - val_accuracy: 0.8572 - val_loss: 0.3159
Epoch 8/50
105/105 ━━━━━━━━━━━━━━━━━━━━ 11s 107ms/step - accuracy: 0.8519 - loss: 0.3099 - val

[I 2025-05-07 13:32:39,636] Trial 0 finished with value: 0.8381846647861058 and parameters: {'num_transformer_blocks': 1, 'head_size': 64, 'num_heads': 4, 'ff_dim': 82, 'dense_units': 37, 'dropout': 0.4385520992095441, 'lr': 0.001450502745324012}. Best is trial 0 with value: 0.8381846647861058.


Epoch 1/50


2025-05-07 13:33:14.984627: W external/local_xla/xla/tsl/framework/bfc_allocator.cc:501] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.78GiB (rounded to 2985397760)requested by op 
2025-05-07 13:33:14.984807: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1058] BFCAllocator dump for GPU_0_bfc
2025-05-07 13:33:14.984820: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1065] Bin (256): 	Total Chunks: 7977, Chunks in use: 7977. 1.95MiB allocated for chunks. 1.95MiB in use in bin. 37.1KiB client-requested in use in bin.
2025-05-07 13:33:14.984824: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1065] Bin (512): 	Total Chunks: 24, Chunks in use: 24. 12.5KiB allocated for chunks. 12.5KiB in use in bin. 9.8KiB client-requested in use in bin.
2025-05-07 13:33:14.984827: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1065] Bin (1024): 	Total Chunks: 40, Chunks in use: 40. 48.2KiB allocated for chunks. 48.2KiB in use in bin. 43.9KiB client-request

ResourceExhaustedError: Graph execution error:

Detected at node StatefulPartitionedCall defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/home/ddias/.local/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/home/ddias/.local/lib/python3.10/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/home/ddias/.local/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/home/ddias/.local/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/home/ddias/.local/lib/python3.10/site-packages/ipykernel/kernelbase.py", line 

In [ ]:
# Get the best hyperparameters
best_transformer_params = transformer_study.best_params
print("Best Transformer hyperparameters:", best_transformer_params)

In [ ]:
transformer_model = build_transformer_model(X_train.shape[1:], num_classes=1, hp=best_transformer_params)
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weight_dict = dict(enumerate(class_weights))

early_stop_final = callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
transformer_model.fit(X_train, y_train, epochs=20, batch_size=BATCH_SIZE, verbose=1,
                      class_weight=class_weight_dict, validation_split=0.2,
                      callbacks=[early_stop_final])

In [ ]:
y_pred_prob = transformer_model.predict(X_test).flatten()
y_pred = (y_pred_prob > 0.5).astype(int)

In [ ]:
test_accuracy = accuracy_score(y_test, y_pred)
test_precision = precision_score(y_test, y_pred)
test_recall = recall_score(y_test, y_pred)
test_f1 = f1_score(y_test, y_pred)
test_auc = roc_auc_score(y_test, y_pred_prob)

print("Transformer Test Accuracy:", test_accuracy)
print("Transformer Test Precision:", test_precision)
print("Transformer Test Recall:", test_recall)
print("Transformer Test F1-score:", test_f1)
print("Transformer Test AUC:", test_auc)

In [ ]:
# Save the model
transformer_model.save('transformer_model.keras')